In [18]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
url = "utterences.csv"
names = ['sentence', 'command']
df = pd.read_csv(url, names=names)

In [20]:
df.head()

,sentence,command
0,tell me the time,time
1,what time is it,time
2,current time,time
3,set temp,set_bed_temp
4,make bed temp,set_bed_temp


In [21]:
df.shape

(59, 2)

In [22]:
df.groupby('command').size()

command
HP_print           3
TR_print           3
bed_temp           3
cancel_print       4
date               2
es_time            2
nozzle_temp        5
p_complete         2
pause_print        4
printer_state      3
restart_print      4
resume_print       5
set_bed_temp       4
set_nozzle_temp    8
start_print        4
time               3
dtype: int64

In [23]:
y = df['command']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [24]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['sentence'])
X = X.todense()
print(vectorizer.get_feature_names())
print(X.shape)
print(X)
y = df['command']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

['abandon', 'be', 'bed', 'been', 'cancel', 'complete', 'completion', 'continue', 'current', 'date', 'estimated', 'extruder', 'going', 'has', 'how', 'initiate', 'is', 'it', 'left', 'long', 'longer', 'make', 'me', 'much', 'nozzle', 'of', 'on', 'pause', 'percent', 'percentage', 'print', 'printer', 'proceed', 'remaining', 'reset', 'restart', 'resume', 'runtime', 'set', 'start', 'state', 'stop', 'tell', 'temp', 'temperature', 'the', 'time', 'to', 'today', 'what', 'will', 'with']
(59, 52)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.46657627 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.45471237 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [25]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.643333 (0.099778)
LDA: 0.693333 (0.077172)


/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/discriminant

KNN: 0.368889 (0.114913)
CART: 0.441111 (0.246496)
NB: 0.621111 (0.104373)
SVM: 0.224444 (0.057263)


In [31]:
# Make predictions on validation dataset
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [27]:
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.25
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 3 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 3 0]
 [0 0 0 0 1 0 0 0]]
                 precision    recall  f1-score   support

       HP_print       0.00      0.00      0.00         1
       TR_print       0.00      0.00      0.00         1
    nozzle_temp       0.00      0.00      0.00         3
  restart_print       0.00      0.00      0.00         2
   resume_print       0.00      0.00      0.00         0
   set_bed_temp       0.00      0.00      0.00         1
set_nozzle_temp       0.43      1.00      0.60         3
           time       0.00      0.00      0.00         1

       accuracy                           0.25        12
      macro avg       0.05      0.12      0.07        12
   weighted avg       0.11      0.25      0.15        12



/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stokesw/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average